# Index Data

Sheet designed to aggregate all the useful data and store it in a database.

In [5]:
import pandas as pd
import hvplot.pandas
import sqlalchemy
from dotenv import load_dotenv
from pathlib import Path
import requests
import json
import os
import time

load_dotenv('data/.env')
AVAPI = os.getenv('Alpha_Vantage_API')

database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

 _______________________

_________________

## SP500 DB Tables

Load DataFrames from the Databases that were constructed below.

### -SP500 Common Stock Shares Outstanding

Shares of common stock oustanding for most of the companies in the S&P 500.  Used in conjunction with closing price to calculate market capitalization.

In [14]:
SP500_CSSO = pd.read_sql_table('SP500 CSSO', engine)
SP500_CSSO.head()

,index,AAPL,GOOGL,GOOG,MSFT,AMZN,FB,JPM,JNJ,XOM,...,FLS,HRB,UA,PWR,AIZ,NAVI,RRC,PDCO,SIG,CHK
0,2013-06-30,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2013-09-30,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2013-12-31,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2014-03-31,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,2014-06-30,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### -SP500 Constituents Adjusted Closing Price

Adjusted Closing Price of most SP500 constituents.  Adjusted price takes dividends and stock plits into account.

In [15]:
SP500_Cons_Adj = pd.read_sql_table('SP500 Cons Adjusted', engine)
SP500_Cons_Adj.head()

,index,AAPL,GOOGL,GOOG,MSFT,AMZN,FB,JPM,JNJ,XOM,...,UA,PWR,CSRA,AIZ,EVHC,NAVI,RRC,PDCO,SIG,CHK
0,1999-11-01,0.593355,NaN,NaN,29.221068,69.13,NaN,28.189088,30.008308,18.842030,...,NaN,18.060058,NaN,NaN,NaN,NaN,2.425223,7.964985,10.762833,NaN
1,1999-11-02,0.613460,NaN,NaN,29.281175,66.44,NaN,28.232943,29.265952,18.683228,...,NaN,18.020568,NaN,NaN,NaN,NaN,2.581689,7.975846,10.762833,NaN
2,1999-11-03,0.623015,NaN,NaN,29.104020,65.81,NaN,27.811254,29.779891,18.400912,...,NaN,18.514192,NaN,NaN,NaN,NaN,2.425223,7.988518,10.762833,NaN
3,1999-11-04,0.639221,NaN,NaN,29.024933,63.06,NaN,28.378004,30.065412,18.400912,...,NaN,20.982313,NaN,NaN,NaN,NaN,2.386420,7.445451,10.973869,NaN
4,1999-11-05,0.675073,NaN,NaN,28.964827,64.94,NaN,29.096563,30.122517,17.800992,...,NaN,20.857261,NaN,NaN,NaN,NaN,2.308187,7.331407,10.813481,NaN


### -SP500 Constituents Closing Price

Closing Price of most SP500 constituents.  This is not adjusted for dividends or stock splits.  Used to find market capitalization, when multiplied by CSSO above.

In [17]:
SP500_Cons_Close = pd.read_sql_table('SP500 Cons Closing', engine)
SP500_Cons_Close.head()

,index,AAPL,GOOGL,GOOG,MSFT,AMZN,FB,JPM,JNJ,XOM,...,UA,PWR,CSRA,AIZ,EVHC,NAVI,RRC,PDCO,SIG,CHK
0,1999-11-01,77.62,NaN,NaN,92.37,69.13,NaN,83.56,105.1,74.75,...,NaN,27.44,NaN,NaN,NaN,NaN,3.875,44.00,25.50,NaN
1,1999-11-02,80.25,NaN,NaN,92.56,66.44,NaN,83.69,102.5,74.12,...,NaN,27.38,NaN,NaN,NaN,NaN,4.125,44.06,25.50,NaN
2,1999-11-03,81.50,NaN,NaN,92.00,65.81,NaN,82.44,104.3,73.00,...,NaN,28.13,NaN,NaN,NaN,NaN,3.875,44.13,25.50,NaN
3,1999-11-04,83.62,NaN,NaN,91.75,63.06,NaN,84.12,105.3,73.00,...,NaN,31.88,NaN,NaN,NaN,NaN,3.813,41.13,26.00,NaN
4,1999-11-05,88.31,NaN,NaN,91.56,64.94,NaN,86.25,105.5,70.62,...,NaN,31.69,NaN,NaN,NaN,NaN,3.688,40.50,25.62,NaN


### -SP500 Constituents

Used as a list to iterate through many times in the worksheets.

In [22]:
SP500_Cons = pd.read_sql_table('SP500_Cons', engine)
SP500_Cons.head()

,index,0
0,0,AAPL
1,1,GOOGL
2,2,GOOG
3,3,MSFT
4,4,AMZN


### -SP500 Constituent Market Capitilization

Calculated off the quarterly common stock shares oustanding and the closing prices.  Quarterly market capitalization of most of hte SP500 stocks.

This is built in the AUMvsMArketCap sheet.

In [12]:
SP500_MC = pd.read_sql_table('SP500_MC', engine)
SP500_MC.head()

,level_0,index,AAPL,GOOG,MSFT,AMZN,FB,JPM,JNJ,XOM,...,MDLZ,COP,GD,CL,GM,ANTM,EOG,AMT,NOC,SYK
0,0,2017-03-31,7.478674e+11,5.741451e+11,5.086368e+11,4.237661e+11,4.106666e+11,3.072959e+11,3.127375e+11,3.743985e+11,...,6.586932e+10,5.416192e+10,5.221558e+10,5.779065e+10,5.304000e+10,3.791896e+10,5.735777e+10,4.513619e+10,4.154491e+10,4.492875e+10
1,1,2017-06-30,7.445520e+11,6.296372e+11,5.313124e+11,4.646400e+11,4.378420e+11,3.163733e+11,3.371070e+11,3.474764e+11,...,6.560561e+10,6.249164e+10,5.649153e+10,6.475119e+10,5.239500e+10,4.383809e+10,5.598736e+10,5.180614e+10,4.470618e+10,4.923710e+10
2,2,2017-09-30,7.900501e+11,6.663800e+11,5.750628e+11,4.633707e+11,4.962065e+11,3.266727e+11,3.561115e+11,3.420530e+11,...,6.127462e+10,6.200237e+10,5.938840e+10,6.551609e+10,5.653200e+10,4.949624e+10,5.199822e+10,5.678843e+10,5.008563e+10,5.190372e+10
3,3,2017-12-31,8.599678e+11,7.270209e+11,6.590857e+11,5.660235e+11,5.134986e+11,3.375992e+11,3.490248e+11,3.473493e+11,...,6.407160e+10,5.437109e+10,6.129265e+10,6.415899e+10,5.738600e+10,4.887601e+10,5.560451e+10,5.866903e+10,5.342862e+10,5.311548e+10
4,4,2018-03-31,8.293839e+11,7.170373e+11,7.018663e+11,7.019599e+11,4.643497e+11,3.663016e+11,3.751203e+11,3.545500e+11,...,6.213597e+10,6.068332e+10,6.040341e+10,6.599620e+10,5.087600e+10,5.762167e+10,6.209077e+10,6.117975e+10,6.088033e+10,5.797210e+10


____________________

_______________________

# Raw Data Sources

The raw data is collected here.

### -Import SP500  Closing Data

from https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview

In [5]:
SP500_Data = pd.read_csv(Path("Data/SP500to2012.csv"), index_col='Date', parse_dates=True, infer_datetime_format=True)
SP500_Data = SP500_Data.astype({'Close':'float64'})
SP500_Data.head()

,Close
Date,
2012-01-31,1312.41
2012-02-01,1324.09
2012-02-02,1325.54
2012-02-03,1344.90
2012-02-06,1344.33


### -Import SP500 Changes

from: https://analyzingalpha.com/sp500-historical-components-and-changes

In [6]:
SP500_Changes = pd.read_csv(Path("Data/SP500_changes.csv"), index_col='date', parse_dates=True, infer_datetime_format=True)
SP500_Changes.head()

,Unnamed: 0,ticker,name,action,cik
date,,,,,
2016-09-08,112,CHTR,Charter Communications,added,NaN
2016-09-08,112,EMC,EMC Corporation,removed,NaN
2016-09-06,113,MTD,Mettler Toledo,added,NaN
2016-09-06,113,TYC,Tyco International,removed,NaN
2012-09-05,208,LYB,LyondellBasell,added,NaN


### -Import ETF AUM DATA

from: https://ycharts.com/dashboard/

In [7]:
ETF_Data = pd.read_csv(Path("Data/ETFAUM.csv"), index_col='Date', parse_dates=True, infer_datetime_format=True)
ETF_Data = ETF_Data.astype({'SPY': 'float64', 'IVV': 'float64', 'VOO': 'float64', 'SP500': 'float64'})
ETF_Data

,SPY,IVV,VOO,SP500
Date,,,,
2021-12-31,455.22,334.71,279.84,40360.0
2020-12-31,329.02,238.85,176.98,31660.0
2019-12-31,307.39,201.41,130.18,26760.0
2018-12-31,243.39,149.37,89.40,21030.0
2017-12-29,277.54,141.49,83.68,22820.0
2016-12-30,224.82,90.60,56.49,19270.0
2015-12-31,182.04,70.35,40.62,17960.0
2014-12-31,215.91,69.69,27.76,18250.0
2013-12-31,174.85,53.70,14.82,16490.0


### -Import Constituent Info
Primarily used for the Tickers, everything else is out of date.

From: https://datahub.io/core/s-and-p-500-companies-financials#resource-constituents-financials

In [8]:
SP500_Constituents_details = pd.read_csv(Path("Data/constituents_financials.csv"), index_col='Symbol')
SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
SP500_Constituents_details.head()

,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
Symbol,,,,,,,,,,,,,
AAPL,Apple Inc.,Information Technology,155.15,16.86,1.579541,9.20,180.10,131.1200,8.095080e+11,7.938600e+10,3.458609,5.66,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOGL,Alphabet Inc Class A,Information Technology,1007.71,31.48,0.000000,22.27,1198.00,824.3000,7.338240e+11,3.421700e+10,6.801692,4.70,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOG,Alphabet Inc Class C,Information Technology,1001.52,40.29,0.000000,22.27,1186.89,803.1903,7.285360e+11,3.271400e+10,6.772653,4.67,http://www.sec.gov/cgi-bin/browse-edgar?action...
MSFT,Microsoft Corp.,Information Technology,85.01,25.76,1.874791,2.97,96.07,63.2200,6.899780e+11,4.107900e+10,7.113097,9.49,http://www.sec.gov/cgi-bin/browse-edgar?action...
AMZN,Amazon.com Inc,Consumer Discretionary,1350.50,296.16,0.000000,6.16,1498.00,812.5000,6.858730e+11,1.613200e+10,3.927053,24.28,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [9]:
SP500_List = SP500_Constituents_details.index.tolist()
ETF_List = ['SPY','IVV','VOO']
# COMMENTED OUT because its long
#SP500_List

In [12]:
# pd.DataFrame(SP500_List).to_sql('SP500_Cons', engine)

### -Import an Individual Stock Ticker

from: Alpha Advantage API https://www.alphavantage.co/documentation/#dailyadj

In [25]:
# This is for Non-Premium
# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey={AVAPI}'
# r = requests.get(url)
# data = r.json()
# COMMENTED OUT because its really long
# print(json.dumps(data, indent=4, sort_keys=True))

# With Premium
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&outputsize=full&apikey={AVAPI}'
r = requests.get(url)
data = r.json()
# COMMENTED OUT because its really long
# print(json.dumps(data, indent=4, sort_keys=True))

In [26]:
df=pd.DataFrame.from_dict(data['Time Series (Daily)'], orient="index")
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2022-02-22,124.2,125.0,122.68,123.92,123.92,5306688,0.0000,1.0
2022-02-18,124.94,125.44,123.61,124.35,124.35,4609212,0.0000,1.0
2022-02-17,128.05,128.5,124.85,124.97,124.97,6796997,0.0000,1.0
2022-02-16,129.45,130.44,128.26,129.18,129.18,4875612,0.0000,1.0
2022-02-15,130.64,131.68,129.61,129.94,129.94,4395714,0.0000,1.0


_______________________

______________________

# Combine Data

Iterate on all SP500 constituents to creaet tables that are stored in the CB.

Much of this seciton is commented out because it takes a lot of time to run and makes ~500 API calls.

### -Aggregate Constituent DF

Create the table, with AAPL, then append the rest of the table with a for loop

from: Alpha Advantage API https://www.alphavantage.co/documentation/#dailyadj

In [28]:
## PULLING ADJUSTED PRICE

def BuildStockColumns(stock):
    # API call and turn into a dataframe
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={stock}&outputsize=full&apikey={AVAPI}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient="index")
    
    # Rename columns and delete unused data, saving dividends in case used in the future
    df.rename(columns={'4. close':'close', '5. adjusted close':'adjusted','7. dividend amount':'dividend','8. split coefficient':'split'}, inplace=True)
    df.drop(columns={'1. open','2. high','3. low','6. volume'}, inplace=True)
    
    # Turn strings into floats
    df = df.astype({'close': 'float64', 'adjusted': 'float64', 'dividend': 'float64', 'split': 'float64'})
    return df

def BuildDBColumnADJ(stock):
    dbdf = BuildStockColumns(stock)
    dbdf.rename(columns={'adjusted': stock}, inplace=True)
    dbdf.drop(columns={'close','dividend','split'}, inplace=True)    
    return dbdf

def BuildDBColumnCLOSE(stock):
    dbdf = BuildStockColumns(stock)
    dbdf.rename(columns={'close':stock}, inplace=True)
    dbdf.drop(columns={'adjusted','dividend','split'}, inplace=True)    
    return dbdf
    

AAPL_df= BuildDBColumnCLOSE('AAPL')

In [29]:
AAPL_df

,AAPL
2022-02-22,164.32
2022-02-18,167.30
2022-02-17,168.88
2022-02-16,172.55
2022-02-15,172.79
...,...
1999-11-05,88.31
1999-11-04,83.62
1999-11-03,81.50
1999-11-02,80.25


This is all commented out because it only needs to be done to build the DB.

In [30]:
# merged_df = AAPL_df

In [31]:
# SP500_index = 100
# target_index = 505
# merged_df.head()

### -Pull all SP500 stocks
This iterates through all the stocks in the list of SP500 stocks and calls the API function.

It prints out what stock its trying to pull and the time so it can be monitored, when an error occurs,
the index can be adjust above and the process restarted, skipping the erroneous stock.

from: Alpha Advantage API https://www.alphavantage.co/documentation/#dailyadj

In [32]:
# # PULLING ADJUSTED
# # It will skip those that do not return an accurate response

# while SP500_index < target_index:
#     index = SP500_index
#     SP500_index+=1
#     print(time.ctime())
#     print(f'Index: {index}, stock: {SP500_List[index]}')
#     try:
#         index_df = BuildDBColumnADJ(SP500_List[index])
#     except:
#         print(f'Skipped: {SP500_List[index]}')
#         continue
#     merged_df = pd.merge(merged_df, index_df, how='outer', left_index=True, right_index=True)
#     # Not Needed with Premium
#     # time.sleep(20)

In [33]:
# # PULLING CLOSING

# while SP500_index < target_index:
#     index = SP500_index
#     SP500_index+=1
#     print(time.ctime())
#     print(f'Index: {index}, stock: {SP500_List[index]}')
#     try:
#         index_df = BuildDBColumnCLOSE(SP500_List[index])
#     except:
#         print(f'Skipped: {SP500_List[index]}')
#         continue
#     merged_df = pd.merge(merged_df, index_df, how='outer', left_index=True, right_index=True)
#     # Not Needed with Premium
#     # time.sleep(20)

This is a record of all the stocks that did not return properly.

Missed Stocks: AGN(99), BBT(138), APC(179), RHT(235), COL(239), CXO(241), MYL(247), WLTW(262), CBS(265), LB(343), COG(406), COG(416), VAR(420), TMK(436), BF.B(491), SCG(496)

In [34]:
# missing_stocks = [99,138,179,235,239,241,247,262,343,406,416,420,436,491,496]
# merged_df

In [35]:
# Used to initialize the table  DO NOT OVERWRITE IT!!!!
# PULLING ADJUSTED
# merged_df.to_sql('SP500 Cons Adjusted', engine)

In [36]:
# Used to initialize the table  DO NOT OVERWRITE IT!!!!
# PULLING CLOSE
# merged_df.to_sql('SP500 Cons Closing', engine)

In [44]:
engine.table_names()

C:\Users\canav\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['SP500 CSSO',
 'SP500 Cons Adjusted',
 'SP500 Cons Closing',
 'SP500_Cons',
 'SP500_MC']

### -Pull Market Capitilization Data

from: Alpha Advantage API https://www.alphavantage.co/documentation/#dailyadj

In [45]:
def BuildCSSOShares(stock):
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={stock}&apikey={AVAPI}'
    response = requests.get(url)
    data = response.json()
    # print(json.dumps(data, indent=4, sort_keys=True))
    stock_dict = {}
    for qrpt in data['quarterlyReports']:
        csso = qrpt['commonStockSharesOutstanding']
        date = qrpt['fiscalDateEnding']
        stock_dict[date] = csso
    stock_df = pd.DataFrame.from_dict(stock_dict, orient='index')
    stock_df.rename(columns={0:stock}, inplace=True)
    return stock_df

Create the initial Dataframe to merge the rest of the stocks.

In [46]:
# Dangerous to activate, resets merged CSSO
# Merged_CSSO = BuildCSSOShares('AAPL')

In [47]:
# CSSO_index = 100
# CSSO_target_index = 505
# Merged_CSSO.head()

In [48]:
# Commented out, only needs to be done once

# while CSSO_index < CSSO_target_index:
#     index = CSSO_index
#     CSSO_index+=1
#     print(time.ctime())
#     print(f'Index: {index}, stock: {SP500_List[index]}')
#     try:
#         index_df = BuildCSSOShares(SP500_List[index])
#     except:
#         print(f'Skipped: {SP500_List[index]}')
#         continue
#     Merged_CSSO = pd.merge(Merged_CSSO, index_df, how='outer', left_index=True, right_index=True)
#     # Not Needed with Premium
#     # time.sleep(20)

In [49]:
# Add to DB, CAREFUL
# Merged_CSSO.to_sql('SP500 CSSO', engine)

In [50]:
engine.table_names()

C:\Users\canav\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['SP500 CSSO',
 'SP500 Cons Adjusted',
 'SP500 Cons Closing',
 'SP500_Cons',
 'SP500_MC']